# von Neumann's addition of SSDE

In [1]:
load('common_computations-ssde.sage')
var('q')

q

First, we construct the automaton from Figure 5 in Heuberger-Prodinger, 2003, Carry propagation in signed digit representations, European J. Combin. 24 (2003), 293–320.

In [2]:
@cached_function
def Figure5_transitions(q):
    (s0, s, s0s, l, lq, lqs) = SR.var('s0 s s0s l lq lqs')
    if greater0(q-3):
        translations = {s0:[0, L, q/2-1], s0s:[0, L], s:[L, q/2-1], l:[q/2+1, H], lq:[q/2+1, H, q], lqs:[H, q]}
    else:
        translations = {s0:[0], s0s:[], s:[], l:[], lq:[2], lqs:[]}
    abbreviated_transitions = [(1, 1, s0, 0), (1, 1, -s0, 0), (1, 4, lq, 0), (1, 5, q/2, 0), (1, 10, -q/2, 0), (1, 9, -lq, 0), (2, 2, q/2, 0),
                 (2, 3, q/2-1, 1), (2, 7, -q/2-1, 1), (2, 8, -q/2, 0), (2, 9, -lqs, 1), (2, 1, s0s, 1), (2, 1, -s, 0),
                 (2, 4, l, 0), (2, 4, q, 1), (7, 7, -q/2, 0), (7, 8, -q/2+1, 1), (7, 2, q/2+1, 1), (7, 3, q/2, 0), (7, 4, lqs, 1),
                 (7, 1, -s0s, 1), (7, 1, s, 0), (7, 9, -l, 0), (7, 9, -q, 1), (3, 1, s0, 0), (3, 1, -s, 1), (3, 2, q/2, 1),
                 (3, 4, l, 1), (3, 4, q, 0), (3, 8, -q/2, 1), (3, 9, -lq, 0), (8, 1, -s0, 0), (8, 1, s, 1), (8, 7, -q/2, 1),
                 (8, 9, -l, 1), (8, 9, -q, 0), (8, 3, q/2, 1), (8, 4, lq, 0), (4, 1, -s, 0), (4, 1, s0s, 0), (4, 2, q/2, 0),
                 (4, 3, q/2-1, 0), (4, 4, lq, 0), (4, 7, -q/2-1, 0), (4, 8, -q/2, 0), (4, 9, -lqs, 0), (9, 1, s, 0),
                 (9, 1, -s0s, 0), (9, 7, -q/2, 0), (9, 8, -q/2+1, 0), (9, 9, -lq, 0), (9, 2, q/2+1, 0), (9, 3, q/2, 0),
                 (9, 4, lqs, 0), (5, 1, s0, 0), (5, 1, -s, 0), (5, 2, q/2, 0), (5, 4, l, 0), (5, 4, q, 0), (5, 8, -q/2, 0),
                 (5, 9, -lq, 0), (10, 1, -s0, 0), (10, 1, s, 0), (10, 7, -q/2, 0), (10, 9, -l, 0), (10, 9, -q, 0),
                 (10, 3, q/2, 0), (10, 4, lq, 0)]
    result = []
    for t in abbreviated_transitions:
        try:
            inputs = translations[t[2]]
        except KeyError:
            try:
                def negative(x):
                    return -x
                inputs = map(negative, translations[-t[2]])
            except KeyError:
                inputs = [t[2]]
        for i in inputs:
            result += [(t[0], t[1], i, t[3])]
    return result

In [3]:
@cached_function
def Figure5(q):
    return Transducer(Figure5_transitions(q), initial_states=[1], final_states=[1, 2, 3, 4, 5, 7, 8, 9, 10])

We look for the special states where solid transitions with input 0 start. 

As the order of the states may change, depending on the operating system, etc., the output may be a different permutation. However, the final results are not influenced.

In [4]:
not_finishing_with_red = [t.from_state.label() for t in Figure5(q).transitions() if t.word_in==[0] and t.word_out==[1]]
not_finishing_with_red # not tested

[2, 7]

In [5]:
Figure5_q = Figure5(q)
Figure5_q.set_coordinates({1:(0,0), 2:(7,7), 3:(7,-7), 4:(4,0), 5:(7,-2), 7:(-7,7), 8:(-7,-7), 9:(-4,0), 10:(-7,-2)})
#view(Figure5_q)

We will use the following sets, defined in the separat file common_computations.sage such that we can combine several transitions between two states:

In [6]:
digits_labels(q)

[-q,
 -H,
 -1/2*q - 1,
 -1/2*q,
 -1/2*q + 1,
 -L,
 0,
 L,
 1/2*q - 1,
 1/2*q,
 1/2*q + 1,
 H,
 q]

DigitwiseSum is the simplification of the weighted automaton $\mathcal{A}^2$ recognizing the digitwise sum of two SSDE.

In [7]:
DigitwiseSum(q)

Transducer with 6 states

Now, we construct the composition $\mathcal{N}_{SSDE}$ of Figure 5 and DigitwiseSum and simplify it by guessing a good partition of the states:

In [8]:
sage.combinat.finite_state_machine.FSMOldProcessOutput=False
@cached_function
def N_SSDE(q):
    N_SSDE_full = Figure5(q)(DigitwiseSum(q))
    assert N_SSDE_full.is_Markov_chain()
    states = sorted(DigitwiseSum(q).states())
    collapse = {7:2, 9:4, 10:5, 8:3, 2:7, 4:9, 5:10, 3:8, 1:1}
    if greater0(q-3):
        collapse2 = {states[0]:states[5], states[1]:states[4], states[2]:states[2], states[3]:states[3], 
                     states[4]:states[1], states[5]:states[0]}
    else:
        collapse2 = {states[0]:states[0], states[1]:states[1], states[2]:states[2]}
    eqclass = set()
    test = True
    for t in N_SSDE_full.transitions():
        found = False
        for s in N_SSDE_full.transitions():
            if collapse[s.from_state.label()[1].label()]==t.from_state.label()[1].label() and \
                    collapse[s.to_state.label()[1].label()]==t.to_state.label()[1].label() and \
                    collapse2[s.from_state.label()[0]]==t.from_state.label()[0] and \
                    collapse2[s.to_state.label()[0]]==t.to_state.label()[0] and \
                    s.word_in[0]==t.word_in[0] and s.word_out==t.word_out:
                found = True
                eqclass.add(frozenset([s.from_state, t.from_state]))
                break
        if not found:
            test = False
            print t      
    classes = list(list(s for s in x) for x in eqclass)
    assert test
    N_SSDE = N_SSDE_full.quotient(classes)
    assert N_SSDE.is_Markov_chain()
    return N_SSDE

In [9]:
N_SSDE(q)

Transducer with 12 states

In [10]:
for s in N_SSDE(q).iter_states(): # not tested
    print s

(((((-1, 1), (1, -1)),), 1),)
(((((0, 0),),), 4), ((((0, 0),),), 9))
(((((0, 1), (1, 0)),), 5), ((((-1, 0), (0, -1)),), 10))
(((((-1, 0), (0, -1)),), 7), ((((0, 1), (1, 0)),), 2))
(((((0, 0),),), 5), ((((0, 0),),), 10))
(((((0, 0),),), 7), ((((0, 0),),), 2))
(((((-1, 0), (0, -1)),), 1), ((((0, 1), (1, 0)),), 1))
(((((0, 1), (1, 0)),), 3), ((((-1, 0), (0, -1)),), 8))
(((((0, 0),),), 1),)
(((((0, 0),),), 3), ((((0, 0),),), 8))
(((((-1, -1),),), 9), ((((1, 1),),), 4))
(((((-1, 0), (0, -1)),), 9), ((((0, 1), (1, 0)),), 4))


Now, we can define the transition matrices and use them to obtain the constants in Theorem 4.

In [11]:
var('z')

z

In [12]:
@cached_function
def B_and_R(q):
    N_SSDE_q = N_SSDE(q)
    n = len(N_SSDE_q.states())
    def matrix_function(t,x):
        result = t.word_in[0]*int(t.word_out==[x])
        if type(result) == sage.structure.factorization.Factorization:
            result = expand(result)
            if result not in QQ:
                result = result.factor()
        return result
    B = z*N_SSDE_q.adjacency_matrix(entry=lambda t: matrix_function(t, 0))
    R = z*N_SSDE_q.adjacency_matrix(entry=lambda t: matrix_function(t, 1))
    assert (B+R)*vector([1]*n) == vector([z]*n)  
    return (B, R)

In [13]:
B, R = B_and_R(q)
for i in range(B.nrows()): # not tested
    for j in range(B.ncols()):
        entry = (B*8*q^2)[i, j]
        if entry:
            print entry.factor()
        else:
            print entry
    print "----" 

0
0
0
0
0
0
0
0
8*q^2*z
0
0
0
----
4*z
2*(q - 4)^2*z
0
16*z
0
8*(q - 3)*z
8*(q - 3)*z
16*z
2*(3*q - 2)*(q - 2)*z
8*(q - 1)*z
4*z
8*(q - 3)*z
----
0
2*(q - 2)*(q - 4)*z
0
8*z
0
8*(q - 2)*z
4*(q - 2)*z
8*z
2*(3*q - 2)*q*z
0
0
4*(q - 2)*z
----
0
2*(q - 2)*(q - 4)*z
0
8*z
0
8*(q - 2)*z
4*(q - 2)*z
8*z
2*(q - 2)*q*z
0
0
4*(q - 2)*z
----
4*z
2*(q - 2)*(q - 4)*z
0
8*z
0
4*(q - 2)*z
8*(q - 2)*z
8*z
2*(3*q^2 - 6*q + 4)*z
4*(q - 2)*z
4*z
8*(q - 2)*z
----
0
(q - 2)*(q - 4)*z
0
8*z
0
4*(q - 2)*z
4*(q - 2)*z
8*z
(3*q - 4)*(q - 2)*z
4*(q - 2)*z
0
4*(q - 2)*z
----
0
2*(q - 2)*(q - 4)*z
16*z
0
8*(q - 2)*z
0
4*(q - 2)*z
0
2*(3*q - 2)*q*z
0
0
4*(q - 2)*z
----
0
0
0
0
0
0
0
0
4*q^2*z
0
0
0
----
4*z
2*(q - 2)*(q - 4)*z
16*z
0
8*(q - 2)*z
0
8*(q - 2)*z
0
2*(3*q^2 - 6*q + 4)*z
0
4*z
8*(q - 2)*z
----
4*z
(q - 2)*(q - 4)*z
0
0
0
0
4*(q - 2)*z
0
(3*q - 2)*q*z
0
4*z
4*(q - 2)*z
----
0
4*(q - 2)*(q - 4)*z
0
0
0
16*(q - 2)*z
0
0
4*(q - 2)*q*z
16*q*z
0
0
----
0
2*(q - 2)*(q - 4)*z
0
8*z
0
8*(q - 2)*z
4*(q - 2)*z
8

In [14]:
for i in range(R.nrows()): # not tested
    for j in range(R.ncols()):
        entry = (R*8*q^2)[i, j]
        if entry:
            print entry.factor()
        else:
            print entry
    print "----" 

0
0
0
0
0
0
0
0
0
0
0
0
----
0
0
0
0
0
0
0
0
0
0
0
0
----
0
0
0
0
0
0
0
0
0
0
0
0
----
0
0
0
0
0
0
0
0
4*(q - 2)*q*z
8*q*z
0
0
----
0
0
0
0
0
0
0
0
0
0
0
0
----
4*z
(q - 4)*(q - 6)*z
0
8*z
0
4*(q - 4)*z
4*(q - 4)*z
8*z
3*(q - 2)*q*z
4*q*z
4*z
4*(q - 4)*z
----
0
0
0
0
0
0
0
0
0
0
0
0
----
0
2*(q - 2)*(q - 4)*z
0
8*z
0
8*(q - 2)*z
4*(q - 2)*z
8*z
2*(q - 2)*q*z
0
0
4*(q - 2)*z
----
0
0
0
0
0
0
0
0
0
0
0
0
----
0
(q - 2)*(q - 4)*z
0
8*z
0
4*(q - 2)*z
4*(q - 2)*z
8*z
(3*q - 4)*(q - 2)*z
4*(q - 2)*z
0
4*(q - 2)*z
----
0
0
0
0
0
0
0
0
0
0
0
0
----
0
0
0
0
0
0
0
0
0
0
0
0
----


The next function computes $G_k$. For a better performance, we switch from the symbolic ring to a multivariate polynomial ring.

We compute the generating function $G_{k}$ by solving a 24-dimensional linear equation with coefficients in FZ. We use the structure of the block matrix $M_k(z)$ to simplify the computation (only 12-dimensional instead of 24-dimensional). The vector is $v=(eF1; eF2)^t$ with $uv = 2 (q+1)^2/(q+2)^2$

In [15]:
@cached_function
def G_k(q1):
    N_SSDE_q = N_SSDE(q1)
    B, R = B_and_R(q1)
    n = B.ncols()
    if q1 in ZZ:
        RZ.<z, b, c> = ZZ[]  #with the additional variables b=(z/q)^k, c=(-z/q^2)^k
        q = q1
    else:
        RZ.<q, z, b, c> = ZZ[]  #with the additional variables b=(z/q)^k, c=(-z/q^2)^k
    FZ = RZ.fraction_field()
    FZM = MatrixSpace(FZ, n)
    FZV = VectorSpace(FZ, n)
    RP = FZM(identity_matrix(n)-R).inverse() - identity_matrix(n)
    assert FZM((RP+identity_matrix(n)) * (identity_matrix(n)-R)) == identity_matrix(n)
    def geometric_k(rational):
        a = var('a')
        rational = SR(deepcopy(rational).subs(z=a))
        part = rational.partial_fraction(a)
        if rational.is_zero():
            pf = [0]
        else:
            pf = list(part.iterator())   
        result = 0
        assert add(pf) == rational
        for frac in pf:
            denom = FZ(FZ(frac.subs(a=z)).denominator())
            if FZ(denom/(q^2+z)) in RZ:
                frac *= (1 + c*z/q^2)
            elif denom/(z-q) in RZ:
                frac *= (1 - b*z/q)
            result += FZ(frac.subs(a=z))
        assert FZ(result.subs(b=0, c=0) - FZ(rational.subs(a=z))) == 0, "%s: %s" \
                %(rational, FZ(result.subs(b=0, c=0)/FZ(rational.subs(a=z))))
        return result
    RPk=matrix.zero(FZ, n, n)
    for i in range(n):
        for j in range(n):
            test = RP[i, j]
            r = geometric_k(test)
            assert r.subs(b=0,c=0) == test
            RPk[i,j] = r
    assert FZM(RPk.subs(b=0, c=0)) == FZM(RP)
    BP = (FZM(identity_matrix(n)-B)).inverse() - identity_matrix(n)
    assert FZM((BP+identity_matrix(n)) * (identity_matrix(n)-B)) == identity_matrix(n)
    BR = BP*RPk
    #initial vector
    e1 = vector([int(s.is_initial) for s in N_SSDE_q.states()])
    #exit vectors
    endweights = {1: 2*(q1+1)/(q1+2), -1: 2*(q1+1)/(q1+2), 0: (q1+1)/(q1+2)}
    eF1 = FZV(vector([endweights[s.label()[0].label()[0].label()[0].label()[0].label()[0]] * \
                      endweights[s.label()[0].label()[0].label()[0].label()[0].label()[1]] \
                      if s.label()[0].label()[1].label() not in not_finishing_with_red else 0 for s in N_SSDE_q.states()]))
    eF2 = (FZV(vector([endweights[s.label()[0].label()[0].label()[0].label()[0].label()[0]] * \
                       endweights[s.label()[0].label()[0].label()[0].label()[0].label()[1]] \
                       for s in N_SSDE_q.states()])) +
           RPk * FZV(vector([endweights[s.label()[0].label()[0].label()[0].label()[0].label()[0]] * \
                             endweights[s.label()[0].label()[0].label()[0].label()[0].label()[1]] \
                             if s.label()[0].label()[1].label() in not_finishing_with_red else 0 \
                             for s in N_SSDE_q.states()]))).subs(b=b*q1/z, c=-c*q1^2/z)
    right_side = FZV(eF1 + BP*eF2)
    x1 = (identity_matrix(n)-BR)\right_side
    assert (identity_matrix(n)-BR)*x1 == right_side
    x2 = eF2 + RPk*x1
    G = e1*x1 + e1*x2 - endweights[0]^2
    return G

In [16]:
G_k(q)

(4*b*q^19*z^2 + 10*b*q^18*z^3 + 2*b*c*q^16*z^4 + 8*b*q^17*z^4 + 3*b*c*q^15*z^5 + 2*b*q^16*z^5 + b*c*q^14*z^6 - 16*q^20*z - 8*b*q^18*z^2 - 16*q^19*z^2 - 30*b*q^17*z^3 - 8*c*q^17*z^3 + 4*q^18*z^3 + 12*b*c*q^15*z^4 - 28*b*q^16*z^4 + 4*q^17*z^4 - 11*b*c*q^14*z^5 - 6*b*q^15*z^5 + 2*c*q^15*z^5 + 2*b*c^2*q^12*z^6 - 19*b*c*q^13*z^6 - 17*b*c^2*q^11*z^7 + b*c*q^12*z^7 - 2*b*c^2*q^10*z^8 - b*c*q^11*z^8 - b*c^2*q^9*z^9 + 32*q^20 + 16*q^19*z + 4*b*q^17*z^2 + 4*c*q^17*z^2 - 40*q^18*z^2 + 14*b*q^16*z^3 - 24*c*q^16*z^3 + 36*q^17*z^3 - 66*b*c*q^14*z^4 - 6*c^2*q^14*z^4 - 24*b*q^15*z^4 + 38*c*q^15*z^4 + 32*q^16*z^4 - 65*b*c*q^13*z^5 + 20*c^2*q^13*z^5 - 3*b*q^14*z^5 + 25*c*q^14*z^5 - 4*q^15*z^5 + 36*b*c^2*q^11*z^6 + 13*b*c*q^12*z^6 + 19*c^2*q^12*z^6 - 21*c*q^13*z^6 + 16*b*c^2*q^10*z^7 + 3*b*c*q^11*z^7 - 11*c^2*q^11*z^7 - 7*b*q^12*z^7 - 4*c*q^12*z^7 + 21*b*c^2*q^9*z^8 - 4*b*c*q^10*z^8 - 4*c^2*q^10*z^8 - 6*b*c^2*q^8*z^9 - b*c*q^9*z^9 - b*c^2*q^7*z^10 + 96*q^19 + 128*q^18*z - 64*b*q^16*z^2 + 48*c*q^16*z^2 - 

For values of q less then qmin, the above derivation is not valid:

In [17]:
qmin

8

In [18]:
G_k(6)

(-1715*z^9*b*c^2 - 228438*z^8*b*c^2 + 95256*z^8*b*c - 115836*z^8*c^2 + 5815908*z^7*b*c^2 - 14193144*z^7*b*c - 6403320*z^7*c^2 - 92684088*z^6*b*c^2 + 4381776*z^7*b - 825552*z^7*c + 463325184*z^6*b*c - 125928432*z^6*c^2 - 372641472*z^6*b - 315487872*z^6*c + 1978657632*z^5*b*c + 61725888*z^5*c^2 + 96018048*z^6 + 4423688640*z^5*b - 13641421248*z^5*c - 8518172544*z^4*b*c + 8086091328*z^4*c^2 - 2592487296*z^5 + 238015024128*z^4*b - 2962842624*z^4*c - 171104161536*z^4 + 2626559986176*z^3*b + 422205073920*z^3*c + 4479818047488*z^3 + 13119467139072*z^2*b + 319987003392*z^2*c + 22399090237440*z^2 - 362865261846528*z - 725730523693056)/(560*z^10*b*c^2 + 84672*z^9*b*c^2 - 30720*z^9*b*c + 30720*z^9*c^2 - 30912*z^8*b*c^2 + 4298112*z^8*b*c + 2139648*z^8*c^2 + 32272128*z^7*b*c^2 - 1596672*z^8*b - 24952320*z^7*b*c + 41499648*z^7*c^2 + 5225472*z^6*b*c^2 + 74649600*z^7*b + 99532800*z^7*c + 551287296*z^6*b*c - 368395776*z^6*c^2 - 31352832*z^7 - 32099328*z^6*b + 4012167168*z^6*c - 559872000*z^5*b*c - 13683

In [19]:
G_k(4)

(375*z^8*b*c^2 - 300*z^8*b*c + 300*z^8*c^2 - 5000*z^7*b*c^2 + 16800*z^7*b*c + 4700*z^7*c^2 + 64000*z^6*b*c^2 - 6400*z^7*b - 5600*z^7*c - 192000*z^6*b*c + 168000*z^6*c^2 + 204800*z^6*b + 171200*z^6*c - 384000*z^5*b*c - 96000*z^5*c^2 - 128000*z^6 - 614400*z^5*b + 9830400*z^5*c + 8601600*z^4*b*c - 3481600*z^4*c^2 + 1536000*z^5 - 40960000*z^4*b - 2048000*z^4*c + 67584000*z^4 - 308019200*z^3*b - 158924800*z^3*c - 786432000*z^3 - 1363148800*z^2*b - 78643200*z^2*c - 3145728000*z^2 + 25165824000*z + 33554432000)/(-135*z^9*b*c^2 + 108*z^9*b*c - 108*z^9*c^2 + 540*z^8*b*c^2 - 5040*z^8*b*c - 2700*z^8*c^2 - 27000*z^7*b*c^2 + 2304*z^8*b + 2016*z^8*c + 24480*z^7*b*c - 56520*z^7*c^2 - 48384*z^7*b - 71136*z^7*c - 389376*z^6*b*c + 271296*z^6*c^2 + 46080*z^7 + 46080*z^6*b - 3248640*z^6*c - 3667968*z^5*b*c + 672768*z^5*c^2 - 599040*z^6 + 6561792*z^5*b + 8183808*z^5*c + 9732096*z^4*b*c - 884736*z^4*c^2 - 25989120*z^5 + 71368704*z^4*b + 75350016*z^4*c + 309657600*z^4 + 783286272*z^3*b - 193462272*z^3*c + 19

In [20]:
G_k(2)

(27*z^5*c + 27*z^4*b*c - 18*z^4*b - 9*z^4*c + 108*z^4 - 36*z^3*b - 126*z^3*c - 324*z^3 - 288*z^2*b - 36*z^2*c - 1080*z^2 + 2592*z + 1728)/(-12*z^6*c - 12*z^5*b*c + 8*z^5*b + 4*z^5*c - 48*z^5 - 16*z^4*b + 88*z^4*c + 144*z^4 + 64*z^3*b - 16*z^3*c + 672*z^3 + 256*z^2*b - 64*z^2*c - 1536*z^2 - 2304*z + 3072)

In [21]:
assert G_k(q).subs(q=2) == G_k(2)
assert G_k(q).subs(q=4) == G_k(4)
assert G_k(q).subs(q=6) == G_k(6)

Now, we decompose the numerator and the denominator like in Theorem 5:

In [22]:
def decomposition(q1):
    R_num = G_k(q1).numerator()
    S_denom = G_k(q1).denominator()
    RQK.<q, z, b, c> = QQ[]
    RQ.<q, z> = QQ[]
    r = dict(((i, j), RQ(RQK(R_num).coefficient({b: i, c: j}))) for i in range(4) for j in range(4))
    s = dict(((i, j), RQ(RQK(S_denom).coefficient({b: i, c: j}))) for i in range(4) for j in range(4))
    assert add([b^i * c^j * r[i, j] for i in range(4) for j in range(4)]) == R_num
    assert add([b^i * c^j * s[i, j] for i in range(4) for j in range(4)]) == S_denom
    s0 = s[0, 0] / (1-z)
    r0 = r[0, 0]
    s1 = s[1, 0]
    s2 = sum([b^i*c^(j-1) *s[i, j] for i in range(4) for j in range(1, 4)])
    assert s2 in RQK
    assert S_denom == (s0*(1-z) + b*s1 + c*s2)
    r1 = sum([b^(i-1)*c^j *r[i, j] for i in range(1, 4) for j in range(4)])
    assert r1 in RQK
    return s0, r0, s1
s0, r0, s1 = decomposition(q)

Checking the condition on the zeros of $s_0$. All solutions should have an absolute value greater than 1.

In [23]:
s0.factor()

(-4) * (q + 1) * (q + z) * (q + 2)^2 * q^7 * (-q^2 + z) * (4*q^7 - q^5*z^2 - 6*q^4*z^2 + 8*q^3*z^2 + 2*q*z^4 - 4*z^4)

In [24]:
solve((s0.factor()[-1][0]).subs(z=sqrt(x))==0, x, solution_dict=True)

[{x: 1/4*(q^5 + 6*q^4 - sqrt(q^4 + 12*q^3 - 12*q^2 - 32*q + 64)*q^3 - 8*q^3)/(q - 2)},
 {x: 1/4*(q^5 + 6*q^4 + sqrt(q^4 + 12*q^3 - 12*q^2 - 32*q + 64)*q^3 - 8*q^3)/(q - 2)}]

This leads to the following inequality, which is always satisfied for $q\geq 2$.

In [25]:
solve(-4*(q-2)+q^5+6*q^4-8*q^3>=0, q), solve((q^4 + 12*q^3 - 12*q^2 - 32*q + 64)*q^6<=(-4*(q-2)+q^5+6*q^4-8*q^3)^2, q)

([[q >= -7.135285913528591]], [[q <= 0.8231527093596059], [q >= 2]])

In [26]:
RZ_single = QQ[z]
RZ_single(s0.subs(q=2, z=RZ_single(z))).complex_roots()

[-2.82842712474619, -2.00000000000000, 2.82842712474619, 4.00000000000000]

Checking $r_0(1)\neq 0$:

In [27]:
RQ_single = QQ[q]
RQ_single(r0.subs(z=1)).real_roots()

[-2.00000000000000,
 -1.00000000000000,
 0.000000000000000,
 0.823152766524283,
 1.00000000000000]

In [28]:
delta = (s1/s0).subs(z=1)
rho = 1
delta.factor()

(1/4) * q^-3 * (q + 1)^-2 * (q - 1) * (4*q^7 - q^5 - 6*q^4 + 8*q^3 + 2*q - 4)^-1 * (4*q^10 + 10*q^9 + 18*q^8 - 4*q^7 - 10*q^6 + 7*q^5 + 44*q^4 - 29*q^3 - 8*q^2 - 20*q + 16)

Checking $\delta>0$:

In [29]:
RQ_single(delta.numerator()).real_roots()

[0.674174884208947, 0.722670076055868, 1.00000000000000]

In [30]:
RQ_single(delta.denominator()).real_roots()

[-1.00000000000000, 0.000000000000000, 0.823152766524283]